In [95]:
import pandas as pd
import librosa 

In [96]:
def count_and_percent(df,category):

    result= pd.DataFrame({
        'count': df[category].value_counts(),
        'percentage': df[category].value_counts(normalize=True)
    })

    return result


In [97]:
def dataset_info(df):
    print(' - 데이터셋 크기: ',len(df))
    print(' - 발화자 수: ', df['id'].nunique())
    print(' - SSD 발화자 수:  ', df[df['disease_type']==1]['id'].nunique())
    print(' - TD 발화자 수:  ', df[df['disease_type']==0]['id'].nunique())
    print('(1) 발음 맞춤/틀림 샘플 분포:')
    print(count_and_percent(df, 'new_label'))
    print('(2) SSD/TD 샘플 분포: ')
    print(count_and_percent(df, 'disease_type'))
    print('(3) 나이 분포: ')
    print(count_and_percent(df, 'age'))

In [98]:
# (1) APAC+KAPP 전체
df = pd.read_csv('/data/selinawisco/kochild/APAC_KAPP_25_dataset.csv') 
dataset_info(df)

 - 데이터셋 크기:  19526
 - 발화자 수:  129
 - SSD 발화자 수:   41
 - TD 발화자 수:   88
(1) 발음 맞춤/틀림 샘플 분포:
           count  percentage
new_label                   
0          15727    0.805439
1           3799    0.194561
(2) SSD/TD 샘플 분포: 
              count  percentage
disease_type                   
0             15653    0.801649
1              3873    0.198351
(3) 나이 분포: 
     count  percentage
age                   
8     4440    0.227389
7     3544    0.181502
3     2627    0.134539
9     2324    0.119021
5     2219    0.113643
6     1940    0.099355
2     1463    0.074926
4      969    0.049626


In [99]:
# (2) 타겟 단어 샘플만 필터
select_target = pd.read_csv('/home/selinawisco/whisper/r04_selected_APAC_KAPP_25_dataset.csv')
print('타겟단어 샘플만 필터링 후:')
dataset_info(select_target)

타겟단어 샘플만 필터링 후:
 - 데이터셋 크기:  9296
 - 발화자 수:  127
 - SSD 발화자 수:   40
 - TD 발화자 수:   87
(1) 발음 맞춤/틀림 샘플 분포:
           count  percentage
new_label                   
0           7352    0.790878
1           1944    0.209122
(2) SSD/TD 샘플 분포: 
              count  percentage
disease_type                   
0              7317    0.787113
1              1979    0.212887
(3) 나이 분포: 
     count  percentage
age                   
8     2245    0.241502
7     1798    0.193417
9     1127    0.121235
5     1113    0.119729
6     1002    0.107788
3      968    0.104131
2      561    0.060349
4      482    0.051850


In [100]:
# (3) whisper/08.1_split_data_5 로 나이와 TD/SSD 그룹 안에서 발화자 ID를 8:2 로 나눔
train = pd.read_csv('/data/selinawisco/kochild/r08_APAC_KAPP_25_train.csv')
test = pd.read_csv('/data/selinawisco/kochild/r08_APAC_KAPP_25_test.csv')

In [101]:
dataset_info(train)

 - 데이터셋 크기:  7041
 - 발화자 수:  96
 - SSD 발화자 수:   30
 - TD 발화자 수:   66
(1) 발음 맞춤/틀림 샘플 분포:
           count  percentage
new_label                   
0           5588    0.793637
1           1453    0.206363
(2) SSD/TD 샘플 분포: 
              count  percentage
disease_type                   
0              5551    0.788382
1              1490    0.211618
(3) 나이 분포: 
     count  percentage
age                   
8     1685    0.239313
7     1426    0.202528
6      872    0.123846
9      856    0.121574
5      787    0.111774
3      740    0.105099
4      353    0.050135
2      322    0.045732


In [102]:
dataset_info(test)

 - 데이터셋 크기:  2255
 - 발화자 수:  32
 - SSD 발화자 수:   11
 - TD 발화자 수:   21
(1) 발음 맞춤/틀림 샘플 분포:
           count  percentage
new_label                   
0           1764    0.782262
1            491    0.217738
(2) SSD/TD 샘플 분포: 
              count  percentage
disease_type                   
0              1766    0.783149
1               489    0.216851
(3) 나이 분포: 
     count  percentage
age                   
8      560    0.248337
7      372    0.164967
5      326    0.144568
9      271    0.120177
2      239    0.105987
3      228    0.101109
6      130    0.057650
4      129    0.057206


In [111]:
# 음성길이

def dur(path):
    return librosa.get_duration(path=path)
df['duration'] = df['audio'].apply(dur)
sorted(df['duration'])

# pd.set_option('display.max_colwidth', None)
print('최대길이: ', df[df['duration']==df['duration'].max()]['duration'].head(1))
print('최소길이: ', df[df['duration']==df['duration'].min()]['duration'].head(1))

최대길이:  3024    6.136916
Name: duration, dtype: float64
최소길이:  11382    0.010499
Name: duration, dtype: float64


In [116]:
# 파일명 표기문제로 중복된 음성
print(len(df))
cols = [col for col in df.columns if col != 'audio'] # 파일 경로만 다르고 나머지가 다 같은 음성을 뽑기위해 오디오 컬럼 외 나머지 컬럼들의 리스트
dups = df.groupby(cols).filter(lambda x: len(x) > 1)  # filter takes in a function that returns a boolean value, so 중복이 일어난 그룹만 필터
dups.sort_values(by='duration')

19526


,audio,disease_type,new_label,gender,id,age,severity,textgrid_text,human_text,target_text,human_text_jamo,target_text_jamo,duration
11382,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,530,8,0,어,어,어,ㅇㅓ,ㅇㅓ,0.010499
11421,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,530,8,0,어,어,어,ㅇㅓ,ㅇㅓ,0.010499
7156,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,410,3,0,빠,빠,빠,ㅃㅏ,ㅃㅏ,0.099229
7073,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,410,3,0,빠,빠,빠,ㅃㅏ,ㅃㅏ,0.099229
7754,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,410,3,0,악,악,악,ㅇㅏㄱ,ㅇㅏㄱ,0.109887
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9675,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,1,male,392,7,0,미끄너틀(미끄럼틀),미끄너틀,미끄럼틀,ㅁㅣㄲㅡㄴㅓㅌㅡㄹ,ㅁㅣㄲㅡㄹㅓㅁㅌㅡㄹ,4.937460
14431,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,949,9,0,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,ㄱㅝㄴㅊㅗㅇㅇㅡㄴ ㅇㅣㄹㅓㅎㄱㅔ ㅈㅓ ㅈㅓ ㅎㅘㅁㅕㄴㅇㅔ ㄴㅏㅇㅗㄴㅡㄴ ㄱㅓㅊㅓㄹㅓ...,ㄱㅝㄴㅊㅗㅇㅇㅡㄴ ㅇㅣㄹㅓㅎㄱㅔ ㅈㅓ ㅈㅓ ㅎㅘㅁㅕㄴㅇㅔ ㄴㅏㅇㅗㄴㅡㄴ ㄱㅓㅊㅓㄹㅓ...,5.084354
14478,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,male,949,9,0,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,권총은 이렇게 저 저 화면에 나오는 거처럼 작은 총을 말하는 거예요,ㄱㅝㄴㅊㅗㅇㅇㅡㄴ ㅇㅣㄹㅓㅎㄱㅔ ㅈㅓ ㅈㅓ ㅎㅘㅁㅕㄴㅇㅔ ㄴㅏㅇㅗㄴㅡㄴ ㄱㅓㅊㅓㄹㅓ...,ㄱㅝㄴㅊㅗㅇㅇㅡㄴ ㅇㅣㄹㅓㅎㄱㅔ ㅈㅓ ㅈㅓ ㅎㅘㅁㅕㄴㅇㅔ ㄴㅏㅇㅗㄴㅡㄴ ㄱㅓㅊㅓㄹㅓ...,5.084354
9679,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,1,male,392,7,0,에이비터(엘리베이터),에이비터,엘리베이터,ㅇㅔㅇㅣㅂㅣㅌㅓ,ㅇㅔㄹㄹㅣㅂㅔㅇㅣㅌㅓ,5.094376


In [ ]:
non_dups = df.drop_duplicates(subset=cols, keep='first') # cols 에 대해 다 같은 row들을 중복으로 간주, 각각 그룹중 첫번째 행만 남기기
non_dups

,audio,disease_type,new_label,gender,id,age,severity,textgrid_text,human_text,target_text,human_text_jamo,target_text_jamo,duration
0,/data/selinawisco/kochild/APAC_25_word/SSD/boy...,1,1,male,275,3,1,푼두(포도)야,푼두,포도,ㅍㅜㄴㄷㅜ,ㅍㅗㄷㅗ,0.679297
1,/data/selinawisco/kochild/APAC_25_word/SSD/boy...,1,1,male,275,3,1,푸두라(포도)야,푸두라,포도,ㅍㅜㄷㅜㄹㅏ,ㅍㅗㄷㅗ,0.934036
2,/data/selinawisco/kochild/APAC_25_word/SSD/boy...,1,0,male,275,3,1,딸기,딸기,딸기,ㄸㅏㄹㄱㅣ,ㄸㅏㄹㄱㅣ,0.600045
3,/data/selinawisco/kochild/APAC_25_word/SSD/boy...,1,1,male,275,3,1,탐탕(사탕),탐탕,사탕,ㅌㅏㅁㅌㅏㅇ,ㅅㅏㅌㅏㅇ,0.600045
4,/data/selinawisco/kochild/APAC_25_word/SSD/boy...,1,0,male,275,3,1,햄,햄,햄,ㅎㅐㅁ,ㅎㅐㅁ,0.452857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19493,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,female,959,9,0,똥,똥,똥,ㄸㅗㅇ,ㄸㅗㅇ,0.589070
19494,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,female,959,9,0,주니니,주니니,주니니,ㅈㅜㄴㅣㄴㅣ,ㅈㅜㄴㅣㄴㅣ,1.374512
19495,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,female,959,9,0,손,손,손,ㅅㅗㄴ,ㅅㅗㄴ,0.981791
19496,/data/selinawisco/kochild/KAPP_25/K_APP_25_wor...,0,0,female,959,9,0,손,손,손,ㅅㅗㄴ,ㅅㅗㄴ,0.897642


In [119]:
dataset_info(non_dups)

 - 데이터셋 크기:  12901
 - 발화자 수:  129
 - SSD 발화자 수:   41
 - TD 발화자 수:   88
(1) 발음 맞춤/틀림 샘플 분포:
           count  percentage
new_label                   
0          10131    0.785288
1           2770    0.214712
(2) SSD/TD 샘플 분포: 
              count  percentage
disease_type                   
0              9031    0.700023
1              3870    0.299977
(3) 나이 분포: 
     count  percentage
age                   
8     2487    0.192776
7     2406    0.186497
3     1849    0.143322
5     1658    0.128517
6     1438    0.111464
9     1309    0.101465
4      879    0.068134
2      875    0.067824
